In [0]:
--CREATE New Schema
CREATE SCHEMA IF NOT EXISTS dev.etl

In [0]:
-- Clone  sample tables
CREATE TABLE IF NOT EXISTS dev.bronze.orders_raw DEEP CLONE samples.tpch.orders;
CREATE TABLE IF NOT EXISTS dev.bronze.customer_raw DEEP CLONE samples.tpch.customer;

In [0]:
select * from dev.etl.orders_agg_gold

In [0]:
-- Add some data incrementally inn Orders
INSERT INTO dev.bronze.orders_raw 
SELECT * FROM samples.tpch.orders
LIMIT 1000

In [0]:
select * from dev.bronze.orders_raw

In [0]:
select * from dev.etl.orders_agg_gold

In [0]:
-- Create Managed Volume
CREATE VOLUME IF NOT EXISTS dev.etl.landing
COMMENT 'Volume for source files'

In [0]:
%python
dbutils.fs.mkdirs("/Volumes/dev/etl/landing/files")
dbutils.fs.mkdirs("/Volumes/dev/etl/landing/autoloader/schemas")

In [0]:
-- Add columns to Source customer_raw
ALTER TABLE dev.bronze.customer_raw
ADD COLUMNS(
  __src_action STRING, -- Possible values I, D, T
  __src_insert_dt TIMESTAMP
)

In [0]:
-- Update the new columns __src_insert_dt and __src_action
UPDATE dev.bronze.customer_raw
SET __src_action = 'I',
    __src_insert_dt = current_timestamp() - INTERVAL '3 days'
;


In [0]:
select * from dev.bronze.customer_raw

In [0]:
select * from dev.etl.customer_scd1_bronze
where c_custkey = 6
;


In [0]:
select * from dev.etl.customer_scd2_bronze
where c_custkey = 6
;

In [0]:
INSERT INTO dev.bronze.customer_raw
VALUES(
  6,'Customer#000000006',	'asdfs23ds2swdgg', 20,	'30-114-968-4951',7638.57,'AUTOMOBILE', 'New Changed Record', 'I', current_timestamp()
)

In [0]:
INSERT INTO dev.bronze.customer_raw
VALUES(
  6,'Customer#000000006',	'fudadsfsf', 20,	'30-114-968-4951',7638.57,'BUILDING', 'Old REcord for back loading', 'I', current_timestamp() - interval '1 days'
)

In [0]:
INSERT INTO dev.bronze.customer_raw
VALUES(
  6,'Customer#000000006',	'asdfs23ds2swdgg', 20,	'30-114-968-4951',7638.57,'AUTOMOBILE', 'New Changed Record', 'D', current_timestamp()
)

In [0]:
INSERT INTO dev.bronze.customer_raw
VALUES(
  null,'Customer#000000006',	'asdfs23ds2swdgg', 20,	'30-114-968-4951',7638.57,'AUTOMOBILE', 'New Changed Record', 'T', current_timestamp()
)

In [0]:
-- Insert data in Orders Raw
-- Setting o_orderestatus other than O, F and P
-- Setting o_orderprice > 0

INSERT INTO dev.bronze.orders_raw
VALUES
(99990, 227285, 'NA', 123023.00, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Demo record 1 for Expectations test'),
(99990, 227285, '0', -100, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Demo record 2 for Expectations test'),
(99990, 227285, null, 999, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Demo record 3 for Expectations test')

In [0]:
-- INSERT Data in Customer RAW
-- Setting c_mktsegment as null
INSERT INTO dev.bronze.customer_raw
VALUES(
    99990,'Customer#000000323',	'4pd3ds2swdgg', 20,	'30-114-968-4957',4638.57,null, 'Demo Record Expectations Test', 'I', current_timestamp()
)

In [0]:
select * from dev.etl.orders_union_bronze 
where o_orderkey = 99999
;

In [0]:
select * from event_log('392f2e44-94ab-467b-929f-1ff7f16e4108')

In [0]:
CREATE VIEW event_log_raw
AS SELECT * FROM event_log('392f2e44-94ab-467b-929f-1ff7f16e4108');

In [0]:
-- Truncate orders raw table
TRUNCATE TABLE dev.bronze.orders_raw;


In [0]:
-- Insert 3 records
INSERT INTO dev.bronze.orders_raw
VALUES
(999, 227285, 'O', 123023.00, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Incremental record'),
(999, 227285, 'O', 100, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Incremental record'),
(999, 227285, 'F', 999, '1992-09-29', '1-URGENT', 'CLERK#000000432',0,'Incremental record')

In [0]:
select * from dev.bronze.orders_raw